In [ ]:
#Importing libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_flops import get_flops

import pathlib
import pandas as pd
import cv2

import time
import os
import shutil
import random as random

In [ ]:
# Settings for the GPU usage
import tensorflow.keras.backend as KB

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config = config)
tf.compat.v1.keras.backend.set_session(session)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices)

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
## Internal Parameters (set according to Table 1)

# NAS
K = [5,5] # Number of stages, (5 = Convolution layers, and 2 = Input and Output)
nDense = 5 # Number of dense FC leayers

Space_Filters = [2**i for i in range(1, 8)]
Space_Kernels = [2*i+1 for i in range(1, 4)]
Space_Activations= ['linear','tanh','sigmoid','relu','leaky_relu'] 
Space_Maxpooling = [2*i for i in range(1, 3)]
Space_Neurons = [2*i for i in range(3, 25)]
Space_Neurons.insert(0,0)
Space_Optimizers = ['RMSprop', 'Adam', 'Adagrad', 'Adadelta']
Space_lr = [10**(-i) for i in range(2,5)]

# QNSA
nIter = 40 # Number of iterations
nPop = 10 # Number of population
nEpoch = 100 # Number of Epochs
pCrossover = 0.9                         # Crossover Percentage
pMutation = 0.1;                          # Mutation Percentage
mu = 0.1;                    # Mutation Rate


In [ ]:
# Dataset
nClass = 3 # Number of classes
INPUTSHAPE = (200,200,3) # Shape of  input images
imgtype = 'rgb' # Type of image

pic_folder_train = "content/train"
pic_folder_valid = "content/valid"
pic_folder_test = "content/test"

# Net parameters
RANDOMROT = 0.5 # 0 means NO
RANDOMFLIP = "horizontal_and_vertical" # 0 means NO

runnum = 30

In [ ]:
# Making Folders

!mkdir HistoryArchive
!mkdir ModelArchive
!mkdir TrainedModel

In [ ]:
# Functions for importing Dataset

def get_all_path(base_path):

    folder_cr = os.path.join(base_path, 'cr')
    folder_sp = os.path.join(base_path, 'sp')
    folder_nd = os.path.join(base_path, 'nd')
    
    img_cr_fname = os.listdir(folder_cr)
    img_sp_fname = os.listdir(folder_sp)
    img_nd_fname = os.listdir(folder_nd)
    
    all_images_paths = []
    for i in range(len(img_cr_fname)):
        all_images_paths.append(os.path.join(folder_cr, img_cr_fname[i]))
    for i in range(len(img_sp_fname)):
        all_images_paths.append(os.path.join(folder_sp, img_sp_fname[i]))
    for i in range(len(img_nd_fname)):
        all_images_paths.append(os.path.join(folder_nd, img_nd_fname[i]))
        
    label_name = ['cr','sp','nd']
    label_to_index = dict((name, index) for index, name in enumerate(label_name))

    all_images_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_images_paths]
    
    return all_images_paths, all_images_labels


def creat_dataset(pic_folder_train, pic_folder_test, pic_folder_valid, target_size, img_mode):
    
    train_images_paths, train_images_labels = get_all_path(pic_folder_train) 
    test_images_paths, test_images_labels = get_all_path(pic_folder_test)
    valid_images_paths, valid_images_labels = get_all_path(pic_folder_valid)
    
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    x_valid = []
    y_valid = []
    
    if img_mode == 'gray':
        for n in range(len(train_images_paths)):
            image_path = train_images_paths[n]
            image_label = train_images_labels[n]
            img = cv2.imread(image_path, flags = 0)
            img = cv2.resize(img, target_size)
            x_train.append(img)
            y_train.append(image_label)

        for n in range(len(test_images_paths)):
            image_path = test_images_paths[n]
            image_label = test_images_labels[n]
            img = cv2.imread(image_path, flags = 0)
            img = cv2.resize(img, target_size)
            x_test.append(img)
            y_test.append(image_label)
        for n in range(len(valid_images_paths)):
            image_path = valid_images_paths[n]
            image_label = valid_images_labels[n]
            img = cv2.imread(image_path, flags = 0)
            img = cv2.resize(img, target_size)
            x_valid.append(img)
            y_valid.append(image_label)
    else:
        for n in range(len(train_images_paths)):
            image_path = train_images_paths[n]
            image_label = train_images_labels[n]
            img = cv2.imread(image_path)
#             img = cv2.resize(img, target_size)
            img = cv2.resize(img, target_size)/255
            x_train.append(img)
            y_train.append(image_label)

        for n in range(len(test_images_paths)):
            image_path = test_images_paths[n]
            image_label = test_images_labels[n]
            img = cv2.imread(image_path)
#             img = cv2.resize(img, target_size)
            img = cv2.resize(img, target_size)/255
            x_test.append(img)
            y_test.append(image_label)
            
        for n in range(len(valid_images_paths)):
            image_path = valid_images_paths[n]
            image_label = valid_images_labels[n]
            img = cv2.imread(image_path)
#             img = cv2.resize(img, target_size)
            img = cv2.resize(img, target_size)/255
            x_valid.append(img)
            y_valid.append(image_label)
            
    x_train = np.array(x_train)
    if img_mode == 'gray':
        x_train = x_train.reshape(x_train.shape[0], target_size[0], target_size[1], 1).astype('float32')/255.
        
    x_test = np.array(x_test)
    if img_mode == 'gray':
        x_test = x_test.reshape(x_test.shape[0], target_size[0], target_size[1], 1).astype('float32')/255.
        
    x_valid = np.array(x_valid)
    if img_mode == 'gray':
        x_valid = x_valid.reshape(x_valid.shape[0], target_size[0], target_size[1], 1).astype('float32')/255.


    y_train = np.array(y_train)
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = np.array(y_test)
    y_test = tf.keras.utils.to_categorical(y_test)
    y_vaid = np.array(y_valid)
    y_valid = tf.keras.utils.to_categorical(y_valid)
    
    return (x_train, y_train), (x_test, y_test), (x_valid, y_valid)


In [ ]:
# Importing dataset
(x_train, y_train), (x_test, y_test), (x_valid, y_valid) = creat_dataset(pic_folder_train, pic_folder_test,pic_folder_valid, INPUTSHAPE[:-1], imgtype)
dataset = ((x_train, y_train), (x_test, y_test), (x_valid, y_valid))
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_valid.shape, y_valid.shape)

from sklearn.utils import shuffle
X_shuffled,y_shuffled = shuffle(x_train, y_train, random_state=0)

In [ ]:
# Data generator

datagen = ImageDataGenerator(rotation_range = 90, horizontal_flip = True, vertical_flip = True)

In [ ]:
# Number of bits

nBits_Arch_list = [0.5*kk*(kk-1) for kk in K]
nBits_Arch = np.sum(nBits_Arch_list)
nBits_Arch = int(nBits_Arch)

nBits_Filters = np.sum(K)+ 2*len(K)
nBits_Filters = int(nBits_Filters)

nBits_Kernels = nBits_Filters 
nBits_Kernels = int(nBits_Kernels)

nBits_MaxP = len(K)
nBits_MaxP = int(nBits_MaxP)

nBits_Neuron = nDense
nBits_Neuron =int(nBits_Neuron)

nBits_Acts = nBits_Filters + nBits_Neuron # one for dense layers
nBits_Acts = int(nBits_Acts)

nBits_Optimizer = 1
nBits_lr = 1

nBits = nBits_Arch + nBits_Filters + nBits_Kernels + nBits_Acts + nBits_MaxP + nBits_Neuron + nBits_Optimizer + nBits_lr

In [ ]:
# list of optimizing variables

BITS_list = [nBits_Arch,nBits_Filters,nBits_Kernels,nBits_Acts,nBits_MaxP,nBits_Neuron,nBits_Optimizer,nBits_lr]
SPACE_list = [Space_Filters, Space_Kernels, Space_Activations, Space_Maxpooling, Space_Neurons, Space_Optimizers,Space_lr]

In [ ]:
# Type of Parameters

Type_List = np.zeros(nBits)
endd = nBits_Arch
Type_List[0: nBits_Arch] = 0 # Binary
Type_List[nBits_Arch:nBits_Arch+ nBits_Filters+ nBits_Kernels] = 1 # Discrete
Type_List[nBits_Arch+nBits_Filters+nBits_Kernels:nBits_Arch+ nBits_Filters+nBits_Kernels+ nBits_Acts] = 2 # Category
Type_List[nBits_Arch+nBits_Filters+nBits_Kernels+nBits_Acts:nBits_Arch+ nBits_Filters+nBits_Kernels+ nBits_Acts+nBits_MaxP+nBits_Neuron] = 1 # Discrete
Type_List[nBits_Arch+ nBits_Filters+nBits_Kernels+ nBits_Acts+nBits_MaxP+nBits_Neuron:nBits_Arch+ nBits_Filters+nBits_Kernels+ nBits_Acts+nBits_MaxP+nBits_Neuron+nBits_Neuron+nBits_Optimizer] = 2 # Category
Type_List[nBits_Arch+ nBits_Filters+nBits_Kernels+ nBits_Acts+nBits_MaxP+nBits_Neuron+nBits_Optimizer:nBits_Arch+ nBits_Filters+nBits_Kernels+ nBits_Acts+nBits_MaxP+nBits_Neuron+nBits_Neuron+nBits_Optimizer+nBits_lr] = 1 # Discrete


In [ ]:
# Upper and lower bounds

Lower = np.zeros((1, nBits))
Lower = Lower.reshape(-1,1)

Upper = [np.ones((1, nBits_Arch))+1,
          len(Space_Filters)*np.ones((1, nBits_Filters)),
          len(Space_Kernels)*np.ones((1, nBits_Kernels)),
          len(Space_Activations)*np.ones((1, nBits_Acts)),
          len(Space_Maxpooling)*np.ones((1, nBits_MaxP)),
          len(Space_Neurons)*np.ones((1, nBits_Neuron)),
          len(Space_Optimizers)*np.ones((1, nBits_Optimizer)),
          len(Space_lr)*np.ones((1, nBits_lr))]
Upper = np.concatenate( Upper, axis=1)
Upper = Upper [0].reshape(-1,1)-1

bounds = np.stack((Lower.T, Upper.T), axis=-1)[0]

In [ ]:
# Caluclating the number of models generated by combination and mutation operators

nCrossover = 2*np.round(pCrossover*nPop/2)  # Number of Parnets (Offsprings)
nMutation = np.round(pMutation*nPop)        # Number of Mutants

sigma = 0.1* (Upper - Lower) #np.ones((1,nBits))  # Mutation Step Size
sigma = sigma.transpose()

In [ ]:
# Converting Bits to lists

def Bit2LISTs(sol,BITS_list,SPACE_list, nClass, K):
  
  nBits_Arch = BITS_list[0]
  nBits_Filters = BITS_list[1]
  nBits_Kernels = BITS_list[2]
  nBits_Acts = BITS_list[3]
  nBits_MaxP = BITS_list[4]
  nBits_Neuron = BITS_list[5]
  nBits_Optimizer = BITS_list[6]
  nBits_lr = BITS_list[7]

  Space_Filters = SPACE_list[0]
  Space_Kernels = SPACE_list[1]
  Space_Activations = SPACE_list[2]
  Space_Maxpooling = SPACE_list[3]
  Space_Neurons = SPACE_list[4]
  Space_Optimizers = SPACE_list[5]
  Space_lr = SPACE_list[6]

  AR = nBits_Arch
  Filters = sol[AR:AR+nBits_Filters]
  F = np.array(Space_Filters)[Filters.astype(int)]


  NK = AR+nBits_Filters
  Kernels = sol[NK:NK+nBits_Kernels]
  KR = np.array(Space_Kernels)[Kernels.astype(int)]

  NA = NK + nBits_Kernels
  Acts = sol[NA:NA+nBits_Acts]
  A = np.array(Space_Activations)[Acts.astype(int)]
  NM = NA+nBits_Acts
  MaxP = sol[NM:NM+nBits_MaxP]
  M = np.array(Space_Maxpooling)[MaxP.astype(int)]

  NN = NM+nBits_MaxP
  Neuron = sol[NN:NN+nBits_Neuron]
  N = np.array(Space_Neurons)[Neuron.astype(int)]
  N[-1] = nClass

  NO = NN+nBits_Neuron
  Optimizer = sol[NO:NO+nBits_Optimizer]
  O = list(np.array(Space_Optimizers)[Optimizer.astype(int)])

  Nlr = NO+nBits_Optimizer
  LearningRate = sol[Nlr:Nlr+nBits_lr]
  LR = list(np.array(Space_lr)[LearningRate.astype(int)])

# Split into different stages
  endF = 0
  endk = 0
  endA = 0

  FF = []
  KK = []
  AA = []

  c = -1
  for j in K:
    c += 1
    # Filters
    startF = endF 
    endF = startF + j + 2
    FF.append (F[startF:endF])

    # Kernels
    startK = endk
    endk = startK + j + 2
    KK.append (KR[startF:endF])

    # Activations
    startA = endA
    endA = startA + j + 2
    if c == len(K)-1:
      endA = endA + nBits_Neuron
    AA.append (list(A[startA:endA]))

  AA[-1][-1] ='softmax'
  LISTs = [FF,KK,M,N,AA,O,LR]
  
  return LISTs

In [ ]:
# Architecture connector and repair

def Bit2SOLMat(K,sol):
  # Number of stages
  nStage = len(K)
  # Separate the architecture
  nBits_Arch_list = [int(0.5*kk*(kk-1)) for kk in K]
  # print(nBits_Arch_list)
  MAT = [None] * len(K)
  start = 0
  end = 0
  c = -1
  for i in nBits_Arch_list:
    c += 1
    end += i

    arch_i = sol[start:end]
    start = end

  # Bit to Mat
    nNode =  K[c]
    mat = np.zeros((nNode+2,nNode+2))

    Index = np.tril_indices(nNode,k=-1)

    In1 = Index [0] + 1
    In2 = Index [1] + 1
    NewIndex = (In1,In2)

    mat[NewIndex] = arch_i

    # Connect
    # if isolated save to next step
    Inps = np.sum(mat,axis = 1)
    Outs = np.sum(mat,axis = 0)
    SumInpOut = Inps + Outs
    Iso = np.where(SumInpOut == 0)[0]
    Isolated = Iso[1:-1] # Except first and last nodes
    
    if (len(Iso) == nNode+2):
      mat[-1,0] = 1
    else:
      # sum  horizontal if zero and not isolated (setdiff) → connect to the first node
      NoInp = np.where(Inps == 0)[0]
      NoInp_ = np.setdiff1d(NoInp, Iso)
      mat[NoInp_,0] = 1

      # sum  vertical if zero  and not isolated (setdiff) → connect to the last node
      NoOut = np.where(Outs == 0)[0]
      NoOut_ = np.setdiff1d(NoOut, Iso)
      mat[-1,NoOut_] = 1

    MAT[c] = mat
  return MAT

In [ ]:
# Generating Deep CNN Architectures (Decoding)

def CNNGenerator(SOLMAT,ACTLIST,FILTERLIST,KERNELIST,K,InputShape,ModelName,MAXPLIST,nNeurons,RANDOMROT,RANDOMFLIP):
    # Definition of connections:
    # Each layer command presents a connection that enters a computation node and then exits.
    # 1) X_i_j comes from node i and enters node j.
    # 2) X_i is the gathering of all connections enering node i; therfore, if we have only one connection to node i, we'll not have this conncetion.
    # 3) X_Input is the first connection  of network.
    # 4) X_Output is the last connection of stages.

  s = -1
  for kk in K: # Each stage

    s += 1 # stage
    print('---------- STAGE = ', s,' ----------')
    solmat = SOLMAT[s]
    ActList = ACTLIST[s]
    FilterList  = FILTERLIST[s]
    KerneList = KERNELIST[s]

    if s == 0:
      exec("X_Input = tf.keras.Input(shape="+str(INPUTSHAPE)+", name='img')")
      print("X_Input = tf.keras.Input(shape="+str(INPUTSHAPE)+", name='img')")

    # Isolated nodes
    Inps = np.sum(solmat,axis = 1)
    Outs = np.sum(solmat,axis = 0)
    SumInpOut = Inps + Outs
    Isolated = np.where(SumInpOut == 0)[0]

    # CNN generation

    print('solmat = ')
    print(solmat)
    for i in np.arange(0,1+K[s]+1):
      # Decline isolated nodes
      if i in Isolated:
        continue

      listofouts =  np.where(solmat[:,i] == 1)[0]
      listofins =  np.where(solmat[i,:] == 1)[0]
      outed = []
      if i == 0:     # X_0 is the input, so we have one input
        for j in listofouts:
          if s == 0:
            string0 = "X_0_"+str(j)+" = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')(X_Input)"
            string = "X_0_"+str(j)+" = ("+prev+")" if i in outed else string0
            outed.append(0)
            prev = "X_0_"+str(j)
            print(string)
            exec(string)
          else:
            string0 = "X_0_"+str(j)+" = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')(X_Output)"
            string = "X_0_"+str(j)+" = ("+prev+")" if i in outed else string0
            outed.append(0)
            prev = "X_0_"+str(j)
            print(string)
            exec(string)

      elif len(listofins)==1: ## if one input, but not the input layer -----------------------------------
        stringInput = str(listofins[0])
        if i == K[s]+1: # Last node of the stage
            string = "X_Output = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')(X_"+stringInput+"_"+str(i)+")"
            print(string)
            exec(string)
        else: # not Last node
          for j in listofouts:
          
            string0 = "X_"+str(i)+"_"+str(j)+" = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')(X_"+stringInput+"_"+str(i)+")"
            string = "X_"+str(i)+"_"+str(j)+" = ("+prev+")" if i in outed else string0
            prev = "X_"+str(i)+"_"+str(j)
            outed.append(i)
            print(string)
            exec(string)

      elif len(listofins)==2: ## if two inputs ----------------------------------
        # Input gathering

        gatheringstring = "X_"+str(i)+" = layers.Concatenate()([X_"+str(listofins[0])+"_"+str(i)+", X_"+str(listofins[1])+"_"+str(i)+"])"
        print(gatheringstring)
        exec(gatheringstring)

        # Output calculation
        stringInput = "X_"+str(i)
        if i == K[s]+1: # Last node of stage
            string = "X_Output = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')("+stringInput+")"
            print(string)
            exec(string)
        else: # not Last node
          for j in listofouts:

            string0 = "X_"+str(i)+"_"+str(j)+" = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')("+stringInput+")"
            string = "X_"+str(i)+"_"+str(j)+" = ("+prev+")" if i in outed else string0
            prev = "X_"+str(i)+"_"+str(j)
            outed.append(i)
            print(string)
            exec(string)

      else: ## if more than two inputs ------------------------------------------
        # Input gathering
        
        gatheringstring = "X_"+str(i)+" = layers.Concatenate()([X_"+str(listofins[0])+"_"+str(i)+", X_"+str(listofins[1])+"_"+str(i)+"])"
        print(gatheringstring)
        exec(gatheringstring)

        for k in listofins[2:]:
          gatheringstring = "X_"+str(i)+" = layers.Concatenate()([X_"+str(k)+"_"+str(i)+", X_"+str(i)+"])" 
          print(gatheringstring)
          exec(gatheringstring)

        # Outout calculation
        stringInput = "X_"+str(i)
        if i == K[s]+1: # Last node of stage
            string = "X_Output = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')("+stringInput+")"
            print(string)
            exec(string)
        else: # not Last node
          for j in listofouts:
            string0 = "X_"+str(i)+"_"+str(j)+" = layers.Conv2D("+str(FilterList[i])+", "+str(KerneList[i])+", activation='"+ActList[i]+"', padding='same')("+stringInput+")"
            string = "X_"+str(i)+"_"+str(j)+" = ("+prev+")" if i in outed else string0
            prev = "X_"+str(i)+"_"+str(j)
            outed.append(i)
            print(string)
            exec(string)

    print('---------- MaxPooling ----------')
    # Maxpooling
    strMaxpooling = "X_Output = layers.MaxPooling2D("+str(MAXPLIST[s])+")(X_Output)"
    exec(strMaxpooling)
    print(strMaxpooling)

  print('---------- Dense ----------')
  # Ending Flatten 
  strBN = "X_Output = layers.BatchNormalization()(X_Output)"
  exec(strBN)
  print(strBN)

  strFlatten= 'X_Output2 = layers.Flatten()(X_Output)'
  exec(strFlatten)
  print(strFlatten)

  # Ending Dense
  for m in np.arange(0,len(nNeurons)):
    i = i + 1
    if nNeurons[m]!=0 :
      nNeuron = nNeurons[m]
      strDense = "X_Output2 = layers.Dense("+str(nNeuron)+", activation = '"+ActList[i]+"')(X_Output2)"
      exec(strDense)
      print(strDense)

  exec("model2 = tf.keras.Model(X_Input, X_Output2, name='my_net')")
  exec("model2.save('ModelArchive/"+ ModelName +".h5')")

In [ ]:
# Removing extra files from previous runs

def RemoveFiles(mypath):
    from os import listdir
    from os.path import isfile, join
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    for f in onlyfiles:
        os.remove(os.path.join(mypath, f))

RemoveFiles('ModelArchive')
RemoveFiles('TrainedModel')
RemoveFiles('HistoryArchive')

In [ ]:
# Checking if the model has already been saved

def CheckAndCost(mysolution,newsol,ModelName):

    cost = 'NotFound'
    
    if not mysolution:
        return cost
    
    for sol in mysolution:
        if np.sum(np.abs(newsol - sol[3]))==0:
            cost = sol[4]
            name = sol[0]
            
            srcpath1 = "HistoryArchive/"+name+".npy"
            dstpath1 = "HistoryArchive/"+ModelName+".npy"
            shutil.copy(srcpath1,dstpath1)
            
            srcpath1 = "ModelArchive/"+name+".h5"
            dstpath1 = "ModelArchive/"+ModelName+".h5"
            shutil.copy(srcpath1,dstpath1)
            
            srcpath1 = "TrainedModel/"+name+".h5"
            dstpath1 = "TrainedModel/"+ModelName+".h5"
            shutil.copy(srcpath1,dstpath1)
            
            return cost
    return cost

In [ ]:
# The objective function

def ObjectiveFunction(sol, ModelName, mysolution, patience):
    KB.clear_session()
    print('**************************** '+ModelName+' ****************************')
    result = CheckAndCost(mysolution,sol,ModelName)
    if result != 'NotFound':
        print('My checking result = '+ str(result))
        return result

    SOLMAT = Bit2SOLMat(K,sol)
    LISTs = Bit2LISTs(sol,BITS_list,SPACE_list, nClass, K)

    [FILTERLIST,KERNELIST,MAXPLIST,nNeurons,ACTLIST,OPTIMIZER,LR] = LISTs
    early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  mode='min',
                                                  patience=patience,
                                                  restore_best_weights= True,
                                                  )
    print('Lists = ')
    print(LISTs)
    CNNGenerator(SOLMAT,ACTLIST,FILTERLIST,KERNELIST,K,INPUTSHAPE,ModelName,MAXPLIST,nNeurons,RANDOMROT,RANDOMFLIP)
    model2 = tf.keras.models.load_model('ModelArchive/'+ModelName+'.h5')
#   keras.utils.plot_model(model2, "arch.png", show_shapes=True, show_layer_activations=True)
    checkpoint_filepath =('TrainedModel/'+ModelName+'.h5') 


  #  It only saves when the model is considered the "best" ( when the val_loss is minumum found so far)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath,
                                                                monitor = 'val_accuracy',
                                                                mode = 'max',
                                                                save_best_only = True)
    lr_decayed_fn = LR[0]

    if OPTIMIZER[0]=='RMSprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr_decayed_fn)

    elif OPTIMIZER[0]=='Adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_decayed_fn)

    elif OPTIMIZER[0]=='Adagrad':
        optimizer = tf.keras.optimizers.Adagrad(learning_rate = lr_decayed_fn)

    elif OPTIMIZER[0]=='Adadelta':
        optimizer = tf.keras.optimizers.Adadelta(learning_rate = lr_decayed_fn)

    model2.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics = ['accuracy'])
    
    history = model2.fit(datagen.flow(X_shuffled, y_shuffled, batch_size=1), epochs=nEpoch, validation_data=(x_valid, y_valid), shuffle=True,callbacks=[model_checkpoint_callback, early_stopping],verbose=1)

    np.save('HistoryArchive/'+ModelName+'.npy',history.history)

    bs = - np.max(history.history['val_accuracy'])

    cp = model2.count_params()

    latency = get_flops(model2, batch_size = 1)
    best_score = np.array([bs,cp,latency])
    print('Objectives = ')
    print(best_score)
    return best_score

In [ ]:
# Loading the model's training history

def LoadHistory(ModelName):
  # PLOT LOSS AND ACCURACY
    %matplotlib inline

    import matplotlib.image  as mpimg
    import matplotlib.pyplot as plt

    
    history=np.load(('HistoryArchive/'+ModelName+'.npy'),allow_pickle='TRUE').item()

    acc=history['accuracy']
    val_acc=history['val_accuracy']
    loss=history['loss']
    val_loss=history['val_loss']

    epochs=range(len(acc)) # Get number of epochs
    return acc, val_acc, loss, val_loss, epochs

In [ ]:
# Functions needed for QNSA

def Dominates(x,y):
  xc = np.array(x['Cost'])
  yc = np.array(y['Cost'])

  b = (xc <= yc).all() and (xc < yc).any()
  return b


def NonDominatedSorting(pop):
  nPop = len(pop)

  for i in range(nPop):
    pop[i]['DominationSet'] = []
    pop[i]['DominatedCount'] = 0

  F = [[]]

  for i in range(nPop):
    for j in range(i+1,nPop):
      p = pop[i].copy()
      q = pop[j].copy()

      if Dominates(p,q):
        p['DominationSet'].append(j)
        q['DominatedCount'] += 1
      if Dominates(q,p):
        q['DominationSet'].append(i)
        p['DominatedCount'] += 1

      pop[i] = p.copy()
      pop[j] = q.copy()

    if pop[i]['DominatedCount'] == 0:
      F[0].append(i)
      pop[i]['Rank'] = 0
  k = 0

  while True:
    Q = []

    for i in F[k]:
      p = pop[i]
      for j in p['DominationSet']:
        q = pop[j]
        q['DominatedCount'] -=1
   
        if q['DominatedCount'] == 0: # Only dominated with F[0] population
          Q.append(j)
          q['Rank'] = k+1

        pop[j] = q
    if not Q:
      break

    F.append(Q)
    k += 1
      
  return pop, F

def CalcCrowdingDistance(pop,F):
  nf = len(F)

  for k in range(nf):

    Costs = []
    for j in F[k]:
      Costs.append(pop[j]['Cost'])

    Costs = np.array(Costs).transpose()

    nobj = Costs.shape[0]
    n = len(F[k])
    d = np.zeros((n,nobj))
    for j in range(nobj):
      c = Costs[j,:]
      cj = (np.sort(c))
      so = (np.argsort(c,kind = 'stable')).astype(int)
      d[so[0],j] = np.inf

      for i in range(1,n-1):
        if cj[i+1] == cj[i-1]:
           d[so[i],j] = 0
        else:
          d[so[i],j] = np.abs(cj[i+1]-cj[i-1]) / (np.abs(cj[0]-cj[-1]))

      d[so[-1],j] = np.inf

    for i in range(n):
      m = F[k][i]
      pop[m]['CrowdingDistance'] = np.sum(d[i,:])

  return pop

def SortPopulation(pop):
  CD = []
  for i in range(len(pop)):
    CD.append(pop[i]['CrowdingDistance'])
  CD = np.array(CD)
  Idx1 = CD.argsort(kind = 'stable')[::-1].astype(int)
  pop = [pop[i] for i in Idx1]

  RS = []
  for i in range(len(pop)):
    RS.append(pop[i]['Rank'])
  RS = np.array(RS)
  Idx2 = RS.argsort(kind = 'stable').astype(int)
  pop = [pop[i] for i in Idx2]

  ranks = [p['Rank'] for p in pop]
  maxranks = np.max(ranks)

  F = []
  ranks = np.array(ranks)

  if maxranks == 0:
    F = [np.arange(len(pop))]
  else:
    for i in range(maxranks+1):
      result = np.where(ranks == i)[0]
      F.append(result)
  return pop, F

def Crossover(x1,x2):
  alpha = np.random.random(len(x1))
  y11 = np.multiply(alpha,x1)
  y12 = np.multiply(1-alpha,x2)
  y1 = y11+y12

  y21 = np.multiply(alpha,x2)
  y22 = np.multiply(1-alpha,x1)
  y2 = y21+y22

  return y1, y2

def Mutate(x, mu, sigma):
  nVar = len(x)
  nMu = np.ceil(mu*nVar).astype(int)

  j = np.random.choice(nVar, nMu, replace = False)
  y = x.copy()
  for i in j:

    y[i] = x[i] + sigma[0][i] * np.random.normal()

  return y

def Prune(x, Type_List):
    n_bin = np.where(Type_List == 0)
    n_one = np.where(x == 1)
    n_prune = np.intersect1d(n_bin, n_one)
    y = x.copy()
    print(n_bin)
    print(n_one)
    print(n_prune)
    if len(n_prune)>0:
        i = np.random.choice(n_prune)
        y[i] = 0
    else:
        i = np.random.choice(n_bin[0])
        y[i] = 1
        
    return y

In [ ]:
# Crossover operators

def Crossover(p1,p2,p3,Q_tensor,Quarter,Type_List):
    
    # pre alloction of children
    c1 = np.zeros_like(p1['Position'])
    c2 = np.zeros_like(p1['Position'])

    # Sort based on ranks
    ranks_list = []
    ranks_list.append(p1['Rank'])
    ranks_list.append(p2['Rank'])
    ranks_list.append(p3['Rank'])
    ranks_list = np.array(ranks_list)
    ranks_ind = ranks_list.argsort(kind = 'stable').astype(int)

    three_pop =[]
    three_pop.append(p1.copy())
    three_pop.append(p2.copy())
    three_pop.append(p3.copy())

    
    Best = three_pop[ranks_ind[0]]['Position'].copy()
    Mid = three_pop[ranks_ind[1]]['Position'].copy()
    Worst = three_pop[ranks_ind[2]]['Position'].copy()
    
    # Choice of actions
    ChosenActions = np.zeros(3) # for each type
    
    for Vartype in range(0,3):
        
        vars_indx = np.where(Type_List == Vartype)[0]
            
        qtable = Q_tensor[Quarter][Vartype]
        arr_softmax = np.exp(qtable) / np.sum(np.exp(qtable))
        # Here we can make some of the zero!
        ChosenActions[Vartype] = int(np.random.choice(range(len(qtable)), p = arr_softmax))
        print('Var = '+str(Vartype)+', Probabilities'+str(arr_softmax)+', Choice = '+str(ChosenActions[Vartype]))
        Best_vars = Best[vars_indx]
        Mid_vars = Mid[vars_indx]
        Worst_vars = Worst[vars_indx]
        
        p1_vars = p1['Position'][vars_indx]
        p2_vars = p2['Position'][vars_indx]
        p3_vars = p3['Position'][vars_indx]

        if ChosenActions[Vartype] == 0: # Action 1: One-point crossover
            c1_vars,c2_vars = one_point_crossover(Best_vars, Mid_vars)
 
        elif ChosenActions[Vartype] == 1: # Action 2: Uniform crossover
            c1_vars,c2_vars = uniform_crossover(Best_vars, Mid_vars)

        elif ChosenActions[Vartype] == 2: # Action 3: Jaya crossover
            c1_vars,c2_vars = jaya_crossover(Best_vars, Mid_vars, Worst_vars)
        
        print(c1_vars)
        print(c2_vars)
        c1[vars_indx] = c1_vars 
        c2[vars_indx] = c2_vars 
    return c1, c2, ChosenActions

def one_point_crossover(arr1, arr2): # 0
    # randomly generate a crossover point
    point = np.random.randint(1, len(arr1))
    
    # perform crossover at the crossover point
    new_arr1 = np.concatenate((arr1[:point], arr2[point:]))
    new_arr2 = np.concatenate((arr2[:point], arr1[point:]))
    
    return new_arr1, new_arr2

 
def uniform_crossover(arr1, arr2): # 3
    p=0.5
    
    mask = np.random.choice([True, False], size=len(arr1), p=[p, 1-p])
    new_arr1 = np.where(mask, arr1, arr2)
    new_arr2 = np.where(mask, arr2, arr1)
    return new_arr1, new_arr2

def jaya_crossover(parent1, parent2, parent3): # 5
 
    # First child                              
    r1 = np.random.random(len(parent2))
    d1 = parent1-np.abs(parent2)

    r2 = np.random.random(len(parent2))
    d2 = parent3-np.abs(parent2)

    t1 = np.multiply(r1,d1)
    t2 = np.multiply(r2,d2)

    offspring1 = np.round(parent2+t1+t2)

    # Second child                              
    r3 = np.random.random(len(parent2))                             
    r4 = np.random.random(len(parent2))

    t3 = np.multiply(r3,d1)
    t4 = np.multiply(r4,d2)

    offspring2 = np.round(parent2+t3+t4)
    
    return offspring1, offspring2

In [ ]:
# QNSA

problem = {
    'CostFunction': ObjectiveFunction,
    'nVar': nBits,
    'VarMin':bounds[:,0],
    'VarMax':bounds[:,1]
}
def QNSA(problem, nIter , nPop):

    # Empty particle
    empty_individual = { 
        'Position': None,
        'Cost': None,
        'Rank': None,
        'DominationSet': None,
        'DominatedCount': None,
        'CrowdingDistance': None,
    }

    # Extract info
    CostFunction = problem['CostFunction']
    nVar = problem['nVar']
    VarMin = problem['VarMin']
    VarMax = problem['VarMax']

    # Initilize global best

    gbest = {'Position':None,
             'Cost':None}

    mysolution = []
    pop = []
    for i in range(0,nPop):
      modelname = 'Model_' + str(-1) + '_' + str(i)
      pop.append(empty_individual.copy())
      pop[i]['Position'] = np.round(np.random.uniform(VarMin, VarMax, nVar))
      pop[i]['Cost']= CostFunction(pop[i]['Position'],modelname,mysolution,patience = 5)
      mysolution.append((modelname, -1, i, pop[i]['Position'], pop[i]['Cost']))
    # Non-Dominated Sorting
    pop, F = NonDominatedSorting(pop)

    # Calculate Crowding Distance
    pop = CalcCrowdingDistance(pop, F)

    # Sort Population
    pop, F = SortPopulation(pop)
    
    # Initialize Q able
    Q_tensor = np.zeros((4,3,3)) # 4 quarters or states, 3  variable types, and 7 action types
    for it in range(nIter):
      patience =5 
      # Crossover
      hnc = nCrossover/2
      popc1 = []
      popc2 = []
      popc = []
      for c in range(int(hnc)):
        popc1.append(empty_individual.copy())
        popc2.append(empty_individual.copy())

      for c in range(int(hnc)):
        rnd = random.sample(range(0,nPop),3)

        p1 = pop[rnd[0]].copy()
        p2 = pop[rnd[1]].copy()
        p3 = pop[rnd[2]].copy()
        
        Quarter = int(4*it/nIter)
        popc1[c]['Position'], popc2[c]['Position'], ChosenActions = Crossover(p1,p2,p3,Q_tensor,Quarter,Type_List)

        popc1[c]['Position'] = np.round(np.maximum(popc1[c]['Position'],VarMin))
        popc1[c]['Position'] = np.round(np.minimum(popc1[c]['Position'],VarMax))

        popc2[c]['Position'] = np.round(np.maximum(popc2[c]['Position'],VarMin))
        popc2[c]['Position'] = np.round(np.minimum(popc2[c]['Position'],VarMax))
        
        modelname = 'Model_' + str(it) + '_' + str(2*c)
        popc1[c]['Cost']= CostFunction(popc1[c]['Position'],modelname,mysolution,patience)
        mysolution.append((modelname, it, 2*c, popc1[c]['Position'], popc1[c]['Cost']))
        
        modelname = 'Model_' + str(it) + '_' + str(2*c+1)
        popc2[c]['Cost']= CostFunction(popc2[c]['Position'],modelname,mysolution,patience)
        mysolution.append((modelname, it, 2*c+1, popc2[c]['Position'], popc2[c]['Cost']))
        
        # Reward
        reward = 0
        
        if Dominates(popc1[0],p1):
            reward += 1
        if Dominates(popc1[0],p2):
            reward += 1
        if Dominates(popc1[0],p3):
            reward += 1
        if Dominates(popc2[0],p1):
            reward += 1
        if Dominates(popc2[0],p2):
            reward += 1
        if Dominates(popc2[0],p3):
            reward += 1

        reward /= 6

        # Update Q tensor
        for Vartype in range(3):
            MaxQ = np.max(Q_tensor[Quarter][Vartype])
            ch = int(ChosenActions[Vartype])
            CurrentQ = Q_tensor[Quarter][Vartype][ch]
            alpha = 1 - 0.9*it/nIter
            gamma = 0.8
            Q_tensor[Quarter][Vartype][ch] = (1-alpha) * CurrentQ + alpha * (reward + gamma * MaxQ)
        print('Q_tensor = ')
        print(Q_tensor[Quarter])
      popc = popc1 + popc2

    # Mutation
      popm = []
      for m in range(int(nMutation)):
        popm.append(empty_individual.copy())

      for k in range(int(nMutation)):

        i1 = np.random.randint(0,nPop)
        p1 = pop[i1].copy()
        
        if np.random.rand()>0.0:
            popm[k]['Position'] = Mutate(p1['Position'], mu, sigma)
        else:
            popm[k]['Position'] = Prune(p1['Position'], Type_List)

        popm[k]['Position'] = np.round(np.maximum(popm[k]['Position'],VarMin))
        popm[k]['Position'] = np.round(np.minimum(popm[k]['Position'],VarMax))

        modelname = 'Model_' + str(it) + '_' + str(2*hnc+k)
        popm[k]['Cost'] = CostFunction(popm[k]['Position'],modelname,mysolution, patience)
        mysolution.append((modelname, it, 2*hnc+k, popm[k]['Position'], popm[k]['Cost']))
        
#     # Merge
      pop = pop + popc + popm

    # #Non-Dominated Sorting
      pop, F = NonDominatedSorting(pop)

    # # Calculate Crowding Distance
      pop = CalcCrowdingDistance(pop, F)

    # # Sort Population
      pop, temp = SortPopulation(pop)

    # # Truncate

      pop = pop[0:nPop]

    # # Non-Dominated Sorting
      pop, F = NonDominatedSorting(pop)

    # # Calculate Crowding Distance
      pop = CalcCrowdingDistance(pop, F)

    # # Sort Population
      pop, F = SortPopulation(pop)

    # # Store F1
      F1 = []
      for f in F[0]:
        F1.append(pop[f]) 
      print('Iteration = ',str(it),'# F1 = ',str(len(F1)))
    return F1, pop, mysolution

In [ ]:
print('Running QNSA  ...')
frontiers, pop, mysolution = QNSA(problem, nIter , nPop)